In [1]:

import os
import langgraph
import langsmith
import numpy as np 
import pandas as pd 

# envirnment variable
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
groq_api_key = os.environ['groq_api_key']
langsmith_api_key = os.environ['langsmith_api_key']

In [3]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]
    

graph_builder = StateGraph(State)

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq()

In [5]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [6]:
graph_builder.add_node("chatbot",chatbot)

In [7]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot",END)

In [8]:
graph = graph_builder.compile()

In [9]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [10]:
def stream_graph_updates(user_inputs: str):
    for event in graph.stream({"messages":[{"role":"user", "content":user_inputs}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)
            
            
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ['quit','exit','q']:
            print("Good bye!")
            break
        stream_graph_updates(user_input)
    except:
        user_input = "What do you know about LangGraph?"
        print("User:" + user_input)
        stream_graph_updates
        break

User:What do you know about LangGraph?
